<a href="https://colab.research.google.com/github/khanabdulmajid/tensorflow_deep_learning/blob/main/06__Transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer Learning with Tensorflow part 3: scaling up

we have seen the power if transfer learning feature extraction and fine tuning , now its time to scale up to all of the classes in food 101(101 toal classes)
Our goal is to beat the original food101 paper with 10% of the training data (leverageing the power of deep learning).

In [19]:
# !wget https://github.com/khanabdulmajid/tensorflow_deep_learning/a90154efbfd5eaf7734bf1e073a94b98d09946dd/helper_functions.py
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-09-03 10:40:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-03 10:40:02 (73.5 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]

--2023-09-03 10:40:02--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.199.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [22]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [21]:
unzip_data("/content/101_food_classes_10_percent.zip")

KeyboardInterrupt: ignored

In [4]:
!mkdir modelcheckpoint

In [23]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import preprocessing

In [24]:
checkpoint_path="/content/modelcheckpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       monitor="val_accuracy",
                                                       save_best_only=True,
                                                       save_weights_only=True)
train_dir="/content/101_food_classes_10_percent/train"
test_dir="/content/101_food_classes_10_percent/test"

In [25]:
IMG_SIZE=(224,224)
train_data_10_percent=preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMG_SIZE
                                                                 )
test_data=preprocessing.image_dataset_from_directory(test_dir,
                                                     label_mode="categorical",
                                                     image_size=IMG_SIZE,
                                                     shuffle=False
                                                     )

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [26]:
data_augmentation=Sequential([
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2)
],  name="data_augmentation")

base_model =tf.keras.applications.EfficientNetB0(include_top=False)

In [27]:
len(train_data_10_percent.class_names)

101

In [28]:
base_model.trainable=False

inputs=tf.keras.layers.Input(shape=(224,224,3),name="input_layer")
x = data_augmentation(inputs)
x = base_model(x,training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name="GLobalbalAvgPooling")(x)
outputs=tf.keras.layers.Dense(len(train_data_10_percent.class_names),activation="softmax",name="output_layer_10")(x)
model_0=tf.keras.Model(inputs,outputs)

In [29]:
model_0.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 GLobalbalAvgPooling (Global  (None, 1280)             0         
 AveragePooling2D)                                               
                                                                 
 output_layer_10 (Dense)     (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-train

In [30]:
model_0.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )

In [18]:

history_model_0=model_0.fit(train_data_10_percent,
                            epochs=5,
                            validation_data=test_data,
                            validation_steps=int(0.15*len(test_data)),
                            callbacks=[checkpoint_callback]
                )

Epoch 1/5
237/237 [==============================] - 1199s 5s/step - loss: 3.3796 - accuracy: 0.2743 - val_loss: 2.4257 - val_accuracy: 0.4725
Epoch 2/5
237/237 [==============================] - 1012s 4s/step - loss: 2.2190 - accuracy: 0.4912 - val_loss: 2.0165 - val_accuracy: 0.5124
Epoch 3/5
237/237 [==============================] - 1015s 4s/step - loss: 1.8464 - accuracy: 0.5564 - val_loss: 1.8994 - val_accuracy: 0.5228
Epoch 4/5
237/237 [==============================] - 985s 4s/step - loss: 1.6365 - accuracy: 0.5978 - val_loss: 1.7600 - val_accuracy: 0.5450
Epoch 5/5
237/237 [==============================] - 972s 4s/step - loss: 1.4906 - accuracy: 0.6286 - val_loss: 1.7173 - val_accuracy: 0.5543


# We have achieved an accuracy of ________. Making using of fine tuning, I will try to improve the model accuracy from here on.

In [31]:
model_0.load_weights(checkpoint_path)

In [32]:
#making some layers trainable in base model
base_model.trainable=True
for layer in base_model.layers[:-10]:
  layer.trainable = False

In [ ]:
initial_epoch=5
fine_tuning_epoch=5+initial_epoch
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=["accuracy"]
                )

fine_tuning_1_history=model_0.fit(train_data_10_percent,
            epochs=fine_tuning_epoch,
            validation_data=test_data,
            validation_steps=int(0.15*len(test_data)),
            initial_epoch=history_model_0.epoch[-1],
            callbacks=[checkpoint_callback]

)

Epoch 5/10
220/237 [==========================>...] - ETA: 50s - loss: 1.4987 - accuracy: 0.6007